In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install dreamsboard -U

### 介绍
使用langchain进行任务规划，构建子任务的会话场景资源，通过MCTS任务执行器，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


## 使用

### 构建任务

- 初始化任务引擎 StructuredTaskStepStoryboard传入需要的任务

- loader_task_step_iter_builder 构建任务的子任务，完成后SimpleTaskStepStore可获取子任务信息

- init_task_engine_dreams 初始化场景加载资源，对子任务进行规划，获取会话的资源信息

```

    os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
    os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
    os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
    os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
    os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
    os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST
```

- init_task_engine_storyboard_executor 构建会话场景执行器，初始化一个会话

- storyboard_code_gen_builder 构建会话场景执行器, 对会话存储进行加载，加载失败重新构建

- generate_step_answer 通过会话场景 获取任务的答案





```

In [2]:
   
from dreamsboard.engine.storage.task_step_store.simple_task_step_store import SimpleTaskStepStore
from langchain_community.chat_models import ChatOpenAI
from dreamsboard.dreams.builder_task_step.base import StructuredTaskStepStoryboard
from dreamsboard.engine.utils import concat_dirs
from dreamsboard.engine.storage.task_step_store.types import DEFAULT_PERSIST_FNAME
from dreamsboard.common.try_parse_json_object import try_parse_json_object
from dreamsboard.engine.memory.mctsr.prompt import RefineResponse
from dreamsboard.dreams.task_step_md.base import TaskStepMD
from dreamsboard.common import _get_assistants_tool

import logging
import os
from dreamsboard.dreams.task_step_to_question_chain.weaviate.prepare_load import get_query_hash
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# 控制台打印
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

logger.addHandler(handler)

/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/dreams/task_step_to_question_chain/weaviate/init_networkx_concept.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML
/mnt/ceph/develop/jiawei/conda_env/weaviate_client/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/ceph/develop/jiawei/conda_env/weaviate_client/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 802: system not yet initialized (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)

In [3]:
os.environ["ZHIPUAI_API_KEY"] = "5fae8f96c5ed49c2b7b21f5c6d74de17.A0bcBERbeZ1gZYoN"

os.environ["OPENAI_API_KEY"] = os.environ.get("ZHIPUAI_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://open.bigmodel.cn/api/paas/v4"
llm = ChatOpenAI(
    openai_api_base='https://open.bigmodel.cn/api/paas/v4',
    model="glm-4-plus",
    openai_api_key=os.environ.get("ZHIPUAI_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
kor_dreams_task_step_llm = ChatOpenAI(
    openai_api_base='https://open.bigmodel.cn/api/paas/v4',
    model="glm-4-plus",
    openai_api_key=os.environ.get("ZHIPUAI_API_KEY"),
    verbose=True,
    temperature=0.95,
    top_p=0.70,
)


tools= [ { "type": "web_search",   "web_search": {"enable": False ,"search_result": False   }}]
llm_with_tools = llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )
kor_dreams_task_step_llm_with_tools = kor_dreams_task_step_llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )

from tests.test_builder_task_step.prompts import (
    AEMO_REPRESENTATION_PROMPT_TEMPLATE as AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST,
    STORY_BOARD_SCENE_TEMPLATE as STORY_BOARD_SCENE_TEMPLATE_TEST,
    STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE as STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST,
    EDREAMS_EVOLUTIONARY_TEMPLATE as EDREAMS_EVOLUTIONARY_TEMPLATE_TEST,
    EDREAMS_PERSONALITY_TEMPLATE as EDREAMS_PERSONALITY_TEMPLATE_TEST,
    DREAMS_GEN_TEMPLATE as DREAMS_GEN_TEMPLATE_TEST,
) 
os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST

cross_encoder_path = "/mnt/ceph/develop/jiawei/model_checkpoint/jina-reranker-v2-base-multilingual"
start_task_context = "什么是损失函数？"
builder = StructuredTaskStepStoryboard.form_builder(
    llm_runable=llm_with_tools,
    kor_dreams_task_step_llm=kor_dreams_task_step_llm_with_tools,
    start_task_context=start_task_context, 
    cross_encoder_path=cross_encoder_path
)

/mnt/ceph/develop/jiawei/conda_env/weaviate_client/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


In [ ]:

# 初始化任务引擎 
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=True)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)
 

In [4]:

from dreamsboard.document_loaders.structured_storyboard_loader import StructuredStoryboard
store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./60f9b7459a7749597e7efa71d1747bc4/storage")
 
structured_storyboard = StructuredStoryboard(json_data=[step.__dict__ for step in list(store_load.task_step_all.values())])

structured_storyboard.parse_table()

,task_step_id,shot_number,scene_number,start_task_context,aemo_representation_context,task_step_name,task_step_description,task_step_level,task_step_question,task_step_question_context,task_step_question_answer,ref_task_step_id
0,5e8636e7-f782-4324-adbc-8c88b825d635,1,story_board0,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,损失函数（Loss Function）,损失函数在机器学习和优化问题中，是一个用来评估模型预测值与实际值之间差异的函数。损失函数的值...,0,### 提出的问题\n\n**问题：** 在机器学习和优化问题中，损失函数是如何具体应用于评...,"[{'ref_id': '64f59fb33fda6d7f0648d313', 'score...",在实际工作中，损失函数的选择和应用直接影响模型的训练效果和最终性能。例如，在推荐系统领域，组...,
1,9a719e13-3c08-48f9-b40f-cb861b1adbd1,2,story_board1,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,"均方误差（Mean Squared Error, MSE）",预测值与实际值之间差的平方的平均值。用于回归问题，如房价预测、股票价格预测等。,0>1,"在回归问题中，均方误差（Mean Squared Error, MSE）是如何通过计算预测值...","[{'ref_id': '62c28ae65aee126c0f8a23c9', 'score...",在进一步探讨MSE的应用时，还需注意其在不同数据分布下的表现。例如，在正态分布数据中，MSE...,
2,0ca4d252-d532-4fe6-84a0-c39c4fb7086f,3,story_board2,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,交叉熵损失（Cross-Entropy Loss）,用于分类问题，特别是多类分类问题。应用于图像分类、文本分类等。,0>2,在图像分类任务中，交叉熵损失函数是如何具体应用于模型的训练过程，以优化模型对多类分类问题的预...,"[{'ref_id': '64af99fe3fda6d7f065a6424', 'score...",在实际应用中，交叉熵损失的优化不仅限于上述方法。例如，在语音识别任务中，通过结合注意力机制和...,
3,14d57a3c-f4f8-4e4e-a5e1-79dadaa290cd,4,story_board3,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,对数损失（Log Loss）,交叉熵损失的一种特例，常用于二分类问题。应用于垃圾邮件检测、疾病诊断等。,0>3,在二分类问题中，对数损失（Log Loss）是如何具体应用于垃圾邮件检测任务的？请详细说明其...,"[{'ref_id': '65b313a0939a5f4082a4ad09', 'score...",在进一步的研究中，我们还探讨了不同权重调整策略对模型性能的影响。例如，动态权重调整策略可以根...,
4,4f4925f9-9388-4e36-a13e-fbf65e93bf3a,5,story_board4,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,hinge损失（Hinge Loss）,主要用于支持向量机（SVM）。应用于分类问题，特别是SVM。,0>4,在支持向量机（SVM）中，hinge损失函数是如何具体应用于分类问题的，并且与其他常见的损失...,"[{'ref_id': '65c5890b939a5f408229913c', 'score...",此外，hinge损失在处理不平衡数据集时也展现出独特优势。例如，在欺诈检测任务中，通过调整正...,
5,247fd778-f99a-4f1f-9e0c-7c79a26d6ef2,6,story_board5,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,Huber损失（Huber Loss）,结合了MSE和绝对误差的优点，对异常值不敏感。应用于回归问题，特别是数据中存在异常值的情况。,0>5,在回归问题中，当数据集中存在异常值时，为什么选择Huber损失（Huber Loss）而不是...,"[{'ref_id': '60d3cf7991e0112ca5d18545', 'score...",在金融风险评估领域，Huber损失的应用尤为显著。通过对比实验发现，与传统均方误差（MSE）...,
6,e5a275a0-8a09-41bc-9cae-4b569dafd4e0,7,story_board6,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,Kullback-Leibler散度（KL Divergence）,衡量两个概率分布之间的差异。应用于变分自编码器（VAE）、生成模型等。,0>6,在变分自编码器（VAE）中，Kullback-Leibler散度（KL Divergence...,"[{'ref_id': '65a123bb939a5f40820344c7', 'score...",。」\n\n在变分自编码器（VAE）中，Kullback-Leibler散度（KL Dive...,
7,0cc2208b-dad0-4816-9975-c9cbe6557b1e,8,story_board7,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,对比损失（Contrastive Loss）,用于度量学习，特别是相似性和差异性度量。应用于人脸识别、图像检索等。,0>7,在人脸识别任务中，对比损失（Contrastive Loss）是如何有效区分相似性和差异性，...,"[{'ref_id': '6449f232582c1376bb223865', 'score...",在实际应用中，对比损失的优化不仅依赖于合理的样本对构造，还需要结合数据增强和模型正则化等技术...,
8,471078eb-1d21-49c6-b4c0-d8180f3924b5,9,story_board8,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,总结,损失函数的选择直接影响模型的训练效果和最终性能。不同的任务类型（如回归、分类、度量学习等）和...,1,**问题**：在构建一个高效且鲁棒的机器学习模型时，如何根据不同的任务类型（如回归、分类、度...,"[{'ref_id': '66ecdacf01d2a3fbfcdfd661', 'score...",在实际操作中，结合具体数据集和模型进行实验，能够更直观地验证不同损失函数的效果。例如，在处理...,
9,1554c556-6769-414e-ad81-12bb8d95317b,10,story_board9,什么是损失函数？,损失函数（Loss Function），在机器学习和优化问题中，是一个用来评估模型预测值与实...,计算机科学研究的相关内容,损失函数的选择和优化也是评估模型性能、改进算法稳定性和容错性的重要环节。通过深入分析文献中的...,1>1,在计算机科学研究中，针对损失函数的选择和优化以评估模型性能、改进算法稳定性和容错性的重要环节...,"[{'ref_id': '62c28ae65aee126c0f8a226e', 'score...",在进一步的研究中，可以考虑将损失函数与模型结构优化相结合，探索更高效的联合优化策略。例如，在...,




#### 场景加载模块设计方案
```

编写符合计算机科学领域的 故事情境提示词，生成研究情境（story_scenario_context），替换现有的langchain会话模板，
1、对这个提示词所要求的输入拆分成子任务，
2、对每个子任务指令转换为子问题，召回问题前3条，
3、对召回内容与问题拼接，合成会话内容变量（scene_content）


对每个子问题相关的召回内容，转换为第一人称的会话总结（研究场景（scene_monologue_context）），

1、对召回内容与问题拼接，对每个前3条组成一个总结任务的提示词，为每个任务标记唯一编号，组成任务上下文（story_board_summary_context）
2、加载编号和story_board_summary_context，转换会话信息
```



#### MCTS设计方案

```
MCT 自优化算法代表了蒙特卡洛树搜索（MCTS）与大型语言模型的结合，将不同的场景执行任务过程抽象为搜索树结构。树上的节点代表当前不同视角下的选择策略，而边表示主体对自身的反思。该算法的操作流程遵循 MCTS 算法的一般模式。

具体来说，我们采用模型的先验知识，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


1、对问题生成的子任务，生成一个合理的规划的节点
2、对每个节点创建一个MCTS任务，
3、输入 problem（总问题的子任务相关的子问题）
4、评分代码重构，将片段摘录器模块集成到一个关于_evaluate_answer逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于evaluate_system_prompt 的生成策略，具体的为编写一个关于带有评估的评估器，由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message

5、自我反思代码重构,将片段摘录器模块集成到一个关于self_refine逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于critic_system_prompt和refine_system_prompt的生成策略，critic_system_prompt为生成一个关于子问题相关的loader_cosplay_builder中自身不完美的评价内容，refine_system_prompt为不完美评价的思考过程和评分值。
具体的为编写一个关于带有评价的生成器和反思生成器，它们由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message
```



#### MCTS执行中的重要环节
```
MCTS中的约束规则如下，需要保证这些节点必须符合下面所定义的基本规则

提示约束：模型在奖励评分期间必须遵守最严格的标准。生成结果需要为JSON Response format
{
    "thought": "The thought process behind the answer.",
    "answer": "A float representing the answer to the problem."
}


高分抑制：评分节点中不存在满分反馈机制；任何超过 95 分的奖励都会按固定金额减少，以遏制过高分数。

重复采样：每次访问搜索树节点都涉及对节点奖励的重复采样，以增强自我评估的可靠性。需要注意的是，当对节点的子节点进行奖励采样时，我们也会对其父节点进行奖励采样，以增加奖励采样的样本量。
```



In [5]:
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
step =0
task_step_store = builder.task_step_store
while not task_engine_builder.empty():
    
    step+=1
    if step<=6 :
        continue
    task_engine = task_engine_builder.get()
    if not task_engine.check_engine_init():
        task_engine.init_task_engine()
        task_engine.init_task_engine_dreams()
        task_engine.init_task_engine_storyboard_executor()

    try:
        code_gen_builder = task_engine.storyboard_code_gen_builder()
        task_step = task_engine.task_step_store.get_task_step(task_engine.task_step_id)
        if task_step.task_step_question_answer is None or len(task_step.task_step_question_answer) == 0:
            task_engine.generate_step_answer(code_gen_builder)
        mcts_node = task_engine.get_mcts_node()
        answer = mcts_node.run()
        
        mcts_node.print()
        print(answer)
        task_step.task_step_question_answer = answer 
        task_step_id = task_engine.task_step_id
        
        task_engine.task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage/{task_step_id}", basename=DEFAULT_PERSIST_FNAME)
        task_engine.task_step_store.persist(persist_path=task_step_store_path) 
        
        task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage", basename=DEFAULT_PERSIST_FNAME)
        task_step_store.persist(persist_path=task_step_store_path) 

    except Exception as e:
        logger.error("场景加载失败", e)


/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/engine/storage/template_store/utils.py:17: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  DATA_KEY: doc.dict(),
/mnt/ceph/develop/jiawei/InterpretationoDreams/src/dreamsboard/dreamsboard/engine/engine_builder.py:214: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  node_without_copy = node.copy()
/mnt/ceph/develop/jiawei/conda_env/weaviate_client/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:460: PydanticDeprecatedSince20: The `dict` method is deprecated; use `

In [6]:

store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./60f9b7459a7749597e7efa71d1747bc4/storage")
task_step_md = TaskStepMD(store_load)
md_text =   task_step_md.format_md() 
from IPython.display import Markdown, display

display(Markdown(md_text.text))


# 什么是损失函数？ 


### 损失函数（Loss Function） [task_id:0](5e8636e7-f782-4324-adbc-8c88b825d635)

损失函数在机器学习和优化问题中，是一个用来评估模型预测值与实际值之间差异的函数。损失函数的值越小，表示模型的预测结果越接近真实值，模型的性能越好。损失函数的选择对于模型的训练效果至关重要，不同的任务和应用场景可能需要不同的损失函数。

均方误差（Mean Squared Error, MSE） [task_id:0>1](9a719e13-3c08-48f9-b40f-cb861b1adbd1)预测值与实际值之间差的平方的平均值。用于回归问题，如房价预测、股票价格预测等。 在进一步探讨MSE的应用时，还需注意其在不同数据分布下的表现。例如，在正态分布数据中，MSE能较好反映模型误差；而在长尾分布数据中，MSE可能被极端值影响，导致评估失真。此时，结合其他指标如MAE（平均绝对误差）和RMSE（均方根误差）进行综合评估，能更全面地反映模型性能。此外，梯度下降法在优化模型参数时，通过调整学习率和迭代次数，可有效降低MSE值，提升模型准确性。在实际应用中，还需根据具体场景灵活调整策略，如在房价预测中，考虑地区经济因素；在股票预测中，结合市场情绪指标，以实现更精准的模型优化。

交叉熵损失（Cross-Entropy Loss） [task_id:0>2](0ca4d252-d532-4fe6-84a0-c39c4fb7086f)用于分类问题，特别是多类分类问题。应用于图像分类、文本分类等。 在实际应用中，交叉熵损失的优化不仅限于上述方法。例如，在语音识别任务中，通过结合注意力机制和交叉熵损失，模型在语音转文本的准确率提升了10%。此外，在推荐系统中，使用交叉熵损失优化用户行为预测模型，显著提高了推荐结果的精准度。这些实例进一步验证了交叉熵损失在不同领域的广泛应用和显著效果。

对数损失（Log Loss） [task_id:0>3](14d57a3c-f4f8-4e4e-a5e1-79dadaa290cd)交叉熵损失的一种特例，常用于二分类问题。应用于垃圾邮件检测、疾病诊断等。 在进一步的研究中，我们还探讨了不同权重调整策略对模型性能的影响。例如，动态权重调整策略可以根据训练过程中的模型表现实时调整样本权重，从而更好地平衡各类样本的关注度。此外，结合L2正则化技术，有效抑制了模型的过拟合现象，进一步提升了模型的泛化能力。

我们还对模型在不同数据集上的表现进行了对比分析，发现优化后的模型在不同数据集上的性能提升具有一致性，验证了优化策略的普适性。通过这些实验，我们不仅验证了当前优化方法的有效性，也为未来的模型优化提供了有价值的参考。

hinge损失（Hinge Loss） [task_id:0>4](4f4925f9-9388-4e36-a13e-fbf65e93bf3a)主要用于支持向量机（SVM）。应用于分类问题，特别是SVM。 此外，hinge损失在处理不平衡数据集时也展现出独特优势。例如，在欺诈检测任务中，通过调整正则化参数和损失权重，SVM结合hinge损失能够更准确地识别少量欺诈行为，显著提升系统的预警能力。具体实验数据显示，在某一金融数据集上，模型的召回率提高了10%，误报率降低了8%。这些实际应用案例和数据结果充分证明了hinge损失在不同领域中的有效性和实用性。

Huber损失（Huber Loss） [task_id:0>5](247fd778-f99a-4f1f-9e0c-7c79a26d6ef2)结合了MSE和绝对误差的优点，对异常值不敏感。应用于回归问题，特别是数据中存在异常值的情况。 在金融风险评估领域，Huber损失的应用尤为显著。通过对比实验发现，与传统均方误差（MSE）相比，使用Huber损失的模型在极端市场波动下的预测误差减少了25%，有效提升了风险评估的准确性。此外，在自动驾驶系统中，Huber损失能够更好地处理传感器数据的噪声和异常值，确保了系统的稳定性和安全性。具体案例显示，在复杂路况下，结合Huber损失的模型比使用MSE的模型平均误差降低了18%，显著提升了驾驶辅助系统的可靠性。

进一步的研究还可以探索Huber损失在医疗诊断、天气预报等领域的应用潜力。例如，在医疗诊断中，Huber损失有助于提高对异常病例的识别精度；在天气预报中，其鲁棒性有助于处理极端天气数据，提升预报的准确性。通过不断优化和扩展Huber损失的应用场景，可以更好地发挥其在各类实际问题中的优势。

Kullback-Leibler散度（KL Divergence） [task_id:0>6](e5a275a0-8a09-41bc-9cae-4b569dafd4e0)衡量两个概率分布之间的差异。应用于变分自编码器（VAE）、生成模型等。 。」

在变分自编码器（VAE）中，Kullback-Leibler散度（KL Divergence）扮演着至关重要的角色，主要用于优化模型以生成高质量样本。以下是具体的作用机制和优化过程：

### 1. VAE的基本原理

VAE是一种基于潜变量模型的生成模型，主要由两部分组成：编码器和解码器。编码器将输入数据 \( \mathbf{x} \) 映射到潜变量空间 \( \mathbf{z} \)，而解码器则将潜变量 \( \mathbf{z} \) 映射回数据空间，生成新的数据样本。

### 2. KL Divergence在VAE中的应用

#### 2.1 变分推断

VAE的核心思想是通过变分推断来近似后验分布 \( p_{\theta}(\mathbf{z} \mid \mathbf{x}) \)。具体来说，引入一个变分分布 \( q_{\phi}(\mathbf{z} \mid \mathbf{x}) \) 来近似这个后验分布。KL Divergence用于衡量这两个分布之间的差异：

\[ D_{\mathrm{KL}}(q_{\phi}(\mathbf{z} \mid \mathbf{x}) \mid\mid p_{\theta}(\mathbf{z} \mid \mathbf{x})) \]

#### 2.2 目标函数

VAE的目标函数是证据下界（ELBO），其表达式为：

\[ \log p_{\theta}(\mathbf{x}) \geq \mathbb{E}_{q_{\phi}(\mathbf{z} \mid \mathbf{x})} \left[ \log \frac{p_{\theta}(\mathbf{x} \mid \mathbf{z}) p(\mathbf{z})}{q_{\phi}(\mathbf{z} \mid \mathbf{x})} \right] \]

其中，\( p_{\theta}(\mathbf{x} \mid \mathbf{z}) \) 是生成模型，\( p(\mathbf{z}) \) 是潜变量的先验分布，通常假设为标准正态分布。ELBO可以分解为两部分：

1. **重构误差**：衡量生成样本与真实样本之间的差异。
2. **KL Divergence**：衡量变分分布与后验分布之间的差异。

### 3. 优化过程

#### 3.1 参数优化

通过最大化ELBO来优化编码器和解码器的参数 \( \theta \) 和 \( \phi \)。具体步骤如下：

1. **编码器**：将输入数据 \( \mathbf{x} \) 映射到潜变量 \( \mathbf{z} \)，得到 \( q_{\phi}(\mathbf{z} \mid \mathbf{x}) \)。
2. **解码器**：从潜变量 \( \mathbf{z} \) 生成数据 \( \mathbf{x} \)，得到 \( p_{\theta}(\mathbf{x} \mid \mathbf{z}) \)。
3. **计算ELBO**：通过采样 \( \mathbf{z} \) 来计算ELBO，并利用梯度下降法更新参数。

#### 3.2 KL Divergence的作用

KL Divergence在优化过程中起到以下作用：

1. **正则化**：通过最小化KL Divergence，可以确保变分分布 \( q_{\phi}(\mathbf{z} \mid \mathbf{x}) \) 尽可能接近真实的后验分布 \( p_{\theta}(\mathbf{z} \mid \mathbf{x}) \)，从而避免过拟合。
2. **平衡重构误差和分布差异**：ELBO中的KL Divergence项与重构误差项相互制衡，确保模型在生成高质量样本的同时，保持潜变量分布的一致性。

### 4. 实际应用

在实际应用中，KL Divergence的具体计算通常通过以下步骤实现：

1. **采样**：从变分分布 \( q_{\phi}(\mathbf{z} \mid \mathbf{x}) \) 中采样 \( \mathbf{z} \)。
2. **计算重构误差**：通过解码器计算 \( p_{\theta}(\mathbf{x} \mid \mathbf{z}) \) 并与真实数据 \( \mathbf{x} \) 计算重构误差。
3. **计算KL Divergence**：通过公式计算 \( D_{\mathrm{KL}}(q_{\phi}(\mathbf{z} \mid \mathbf{x}) \mid\mid p_{\theta}(\mathbf{z} \mid \mathbf{x})) \)。
4. **更新参数**

对比损失（Contrastive Loss） [task_id:0>7](0cc2208b-dad0-4816-9975-c9cbe6557b1e)用于度量学习，特别是相似性和差异性度量。应用于人脸识别、图像检索等。 在实际应用中，对比损失的优化不仅依赖于合理的样本对构造，还需要结合数据增强和模型正则化等技术。例如，通过数据增强可以增加样本的多样性，从而提高模型的泛化能力。此外，对比损失还可以与其他损失函数结合使用，如三元组损失（Triplet Loss），进一步提升模型的性能。具体实现时，可以通过调整边际参数和样本对的选择策略，找到最佳的模型配置。实验表明，合理的对比损失设置能够显著提升人脸识别系统的准确率和鲁棒性，尤其在复杂环境下表现出色。

### 总结 [task_id:1](471078eb-1d21-49c6-b4c0-d8180f3924b5)

损失函数的选择直接影响模型的训练效果和最终性能。不同的任务类型（如回归、分类、度量学习等）和应用场景（如异常值处理、概率分布比较等）需要选择合适的损失函数。理解各种损失函数的特性和适用场景，对于构建高效、鲁棒的机器学习模型至关重要。

计算机科学研究的相关内容 [task_id:1>1](1554c556-6769-414e-ad81-12bb8d95317b)损失函数的选择和优化也是评估模型性能、改进算法稳定性和容错性的重要环节。通过深入分析文献中的研究成果，可以更好地理解不同损失函数在特定任务中的应用价值和创新性。 在进一步的研究中，可以考虑将损失函数与模型结构优化相结合，探索更高效的联合优化策略。例如，在深度学习模型中，通过动态调整损失函数权重，自适应地优化模型在不同训练阶段的性能。此外，针对特定任务如自然语言处理中的序列生成任务，可以设计任务特定的损失函数，如结合注意力机制的损失函数，以提升模型的生成质量和语义一致性。

在实验验证方面，除了常用的数据集和评估指标，还可以引入跨领域数据集和多任务学习场景，验证损失函数在不同应用环境下的泛化能力。通过多角度的实验验证，可以更全面地评估损失函数的有效性和适用性。


 



StructuredTaskStepStoryboard
    
对任务进行规划，生成段落之间组成一个动态上下文，  扩写任务步骤构建MCTS任务
    输入：
    	start_task_context： 起始任务
    	llm： 模型
    	kor_dreams_task_step_llm: 任务抽取模型
    	task_step_store: 任务存储器（SimpleTaskStepStore）
    	cross_encoder_path: ranking模型路径（sentence_transformers），当前设计耦合了业务，之后会单独设计一个召回模块

    任务：

        1、对任务（AEMO_REPRESENTATION_PROMPT_TEMPLATE）按照提示词要求进行扩写，将扩写任务步骤收集 （src/dreamsboard/dreamsboard/engine/entity/task_step、src/dreamsboard/tests/test_kor/test_kor3.py）

        2、收集每个任务后存储到磁盘（src/dreamsboard/dreamsboard/engine/storage/task_step_store）

        3、对每个子任务载入会话场景，然后按照扩写任务步骤构建，MCTS任务 loader_task_step_iter_builder


	》 TaskEngineBuilder 场景加载模块
		执行会话场景资源初始化，构建MCTS任务

    根据任务步骤，构建场景加载模块，生成资源文件csv
    根据每个任务，载入StructuredDreamsStoryboard 会话场景
    按照扩写任务步骤构建MCTS任务

		输入：
			task_step_id
			task_step_store: 任务存储器（SimpleTaskStepStore）
			start_task_context： 起始任务
			llm： 模型

		任务：
			init_task_engine：
				初始化任务引擎
        》TaskStepToQuestionChain 
        	输入：
        		client： 矢量库客户端
        		llm： 模型
					invoke_task_step_to_question：1、 对开始任务进行抽取，得到任务步骤，提示词所要求的输入拆分成子任务， 
					invoke_task_step_question_context： 2、对每个子任务指令转换为子问题，召回问题前3条，对任务步骤进行抽取，得到任务步骤的上下文
					export_csv_file_path: 3、对召回内容与问题 导出csv文件

			init_task_engine_dreams
				初始化场景加载资源
					StoryBoardDreamsGenerationChain
					对每个子任务通过职业提示词，载入会话场景
						1、构建场景信息（story_scenario_context），提示词（STORY_BOARD_SCENE_TEMPLATE）
						2、对任务上下文(story_board_summary_context)，构建第一人称数据(scene_monologue_context),提示词（STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE）
						3、对任务上下文(story_board_summary_context)，获取任务分析(evolutionary_step), 提示词（EDREAMS_EVOLUTIONARY_TEMPLATE）
						4、对任务分析(evolutionary_step)，分析对话预设信息（性格）， 提示词（EDREAMS_PERSONALITY_TEMPLATE）
						5、对任务上下文(story_board_summary_context)，场景信息story_scenario_context, 第一人称数据(scene_monologue_context)，
						生成关于人物职业的引导话术，提示词（DREAMS_GEN_TEMPLATE）

			init_task_engine_storyboard_executor

				构建会话场景执行器 StructuredDreamsStoryboard
					对剧本和分析结果进行结构化，将开放问题与性格分析结果进行结合。生成情景扮演会话场景执行器
			    此过程如下
			        对开放问题结果进行抽取，得到问题内容
			        对性格分析结果进行抽取，得到性格分析结果
			        增加系统提示词
			        根据剧本与任务性格基础情景扮演代码，根据每步的抽取析得到的问题，生成问题的答案
			       	在上下文中增加，关于人物职业的引导话术
	        		导出会话场景执行器
					
	    storyboard_code_gen_builder
	    	构建会话场景执行器

	    generate_step_answer
	    	获取主进程子任务的答案
	    
	    get_mcts_node
	    	构建MCTS树, 初始化当前任务相关的MCTS节点，并返回MCTS执行器
